In [158]:
import string

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import spacy 
import praw
import streamlit as st


%matplotlib inline

In [44]:

reddit = praw.Reddit(client_id='qQBQxY9R1zGrYJH9pACBOw', client_secret='MjrjCout_Omkh_3feZFq3UrzyLBicw', user_agent='Crypto')


In [45]:
st.title('Crypto Trader Assistant')
st.text('This tool gives traders and investors a sentiment about the cryptocurrency from r/Cryptocurrency.')

ucurr = st.text_input('Enter a word related to the Cryptocurrency world:', 'e.g. btc')
st.balloons()


In [47]:
crypto = []
ml_subreddit = reddit.subreddit('Cryptocurrency')
for post in ml_subreddit.hot(limit=30000):
    crypto.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created])
crypto = pd.DataFrame(crypto,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created'])

In [48]:
crypto.dropna(subset=['body'], inplace=True)

In [49]:
crypto['original_body'] = crypto['body']

In [50]:
import spacy
nlp = spacy.blank('en')


def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

def remove_stop_words(text):
    doc = nlp(text)
    nlp.Defaults.stop_words |= {'nt','crypto', 'cryptocurrency', ' nt', 'nt '}
    return " ".join([token.text for token in doc if not token.is_stop])

def lemmatize_words(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc])

remove_spaces = lambda x : re.sub('\\s+', ' ', x)

# Reference : https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b
def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

remove_double_quotes = lambda x : x.replace('"', '')
remove_single_quotes = lambda x : x.replace('\'', '')
trim = lambda x : x.strip()

other_chars = ['*', '#', '&x200B', '[', ']', '; ',' ;' "&nbsp", "“","“","”", "x200b"]


def remove_other_chars(x: str):
    for char in other_chars:
        x = x.replace(char, '')
    
    return x


def lower_case_text(text):
    return text.lower()

funcs = [
    remove_urls, 
    lower_case_text,
    remove_punctuation,
    remove_stop_words, 
    remove_emoji, 
    remove_double_quotes, 
    remove_single_quotes,
    remove_other_chars,
    lemmatize_words,
    remove_spaces,
    trim]



In [51]:
for fun in funcs:
    crypto['body'] = crypto['body'].apply(fun)

In [52]:
crypto.reset_index(inplace=True)
crypto.drop(['index'], axis=1, inplace=True)



In [53]:
body_list = crypto.body.tolist()

In [54]:
def generate_ngrams(text, n_gram=2):
    token = [token for token in text.lower().split(' ') if token != '']
    ngrams = zip(*[token[i:] for i in range(n_gram)])
    return [' '.join(ngram) for ngram in ngrams]

In [56]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import linalg
from sklearn import decomposition
import fbpca

number_of_topics = 30
num_top_words = 20
vectorizer = TfidfVectorizer()

In [57]:
bigram_vectorizer = TfidfVectorizer(ngram_range=(2,2))

In [58]:
bigrams_vectors = bigram_vectorizer.fit_transform(body_list).todense()
bigrams_vocab = np.array(bigram_vectorizer.get_feature_names())

# LDA

In [59]:
docs = [generate_ngrams(body) for body in body_list]

In [61]:
from gensim.corpora import Dictionary

dic = Dictionary(docs)

2021-11-20 13:06:56.628 adding document #0 to Dictionary(0 unique tokens: [])
2021-11-20 13:06:56.701 built Dictionary(36632 unique tokens: ['2fa possible', 'acting trade', 'actions share', 'afford lose', 'age requirements']...) from 635 documents (total 41941 corpus positions)
2021-11-20 13:06:56.701 Dictionary lifecycle event {'msg': "built Dictionary(36632 unique tokens: ['2fa possible', 'acting trade', 'actions share', 'afford lose', 'age requirements']...) from 635 documents (total 41941 corpus positions)", 'datetime': '2021-11-20T13:06:56.701917', 'gensim': '4.1.2', 'python': '3.8.8 (default, Apr 13 2021, 12:59:45) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'created'}


In [62]:
corpus = [dic.doc2bow(doc) for doc in docs]

In [63]:
from gensim.models import LdaModel

model = LdaModel(corpus=corpus, id2word=dic, num_topics=number_of_topics, chunksize=2500, passes=5, random_state=1)

2021-11-20 13:07:10.132 using symmetric alpha at 0.03333333333333333
2021-11-20 13:07:10.134 using symmetric eta at 0.03333333333333333
2021-11-20 13:07:10.144 using serial LDA version on this node
2021-11-20 13:07:10.236 running online (multi-pass) LDA training, 30 topics, 5 passes over the supplied corpus of 635 documents, updating model once every 635 documents, evaluating perplexity every 635 documents, iterating 50x with a convergence threshold of 0.001000
2021-11-20 13:07:10.237 too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2021-11-20 13:07:10.581 -63.361 per-word bound, 11846227624940990464.0 perplexity estimate based on a held-out corpus of 635 documents with 41941 words
2021-11-20 13:07:10.582 PROGRESS: pass 0, at document #635/635
2021-11-20 13:07:10.846 topic #16 (0.033): 0.002*"24 hours" + 0.001*"ve seen" + 0.001*"wo nt" + 0.001*"limit orders" + 0.001*"limit order" + 0.001*"total monetary" + 0.001*"e

In [139]:
x = model.show_topics(num_topics=30, num_words=20,formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

tid = np.nan
flag = 0
for topic,words in topics_words:
    print(str(topic)+ ":"+ str(words))
    for word in words:
        if len(re.findall('('+'btc'+')', word)):
            tid = topic
            flag = 1
            break
    if flag:
        break
        
      

0:['crypto works', 'm curious', 'huge fan', 'current market', 'investing crypto', 'wo nt', 'invest based', 'market rebound', 'nt exclude', 'crypto people', 'people nt', 'feel like', 'trading crypto', 'want invest', 'new crypto', 'language function', 'adventurer subscription', 'works getting', 'crypto language', 'experience new']
1:['cash rewards', 'social media', 'crypto daily', 'airdrops giveaways', 'dip nt', 'defi platforms', 'defi staking', 'earn crypto', 'earn programs', 'learn earn', 'btc like', 'dex launched', 'blah blah', 'm right', 'maiar dex', 'time high', 'hedge funds', 'like know', '4 cash', 'debit cards']


In [140]:
if np.isnan(tid):
     st.error('Topic not found, please revise the spelling or enter another topic.')
else:
     ilist = []
     for doc in range(len(docs)):
          prob = model.get_document_topics(dic.doc2bow(docs[doc]),minimum_probability=0)
          if prob[tid][1] > 0.15:
               ilist.append(doc)
               
print(ilist)

[77, 153, 160, 171, 264, 272, 333, 364, 506, 527, 535, 547, 550, 578, 628]


In [144]:
from textblob import TextBlob
# create a function to get subjectivity
def getSubjectivity(twt):
    return TextBlob(twt).sentiment.subjectivity

# create a function to get the polarity
def getPolarity(twt):
    return TextBlob(twt).sentiment.polarity

# create two new columns called "Subjectivity" & "Polarity"
crypto['subjectivity'] = crypto['original_body'][ilist].apply(getSubjectivity)
crypto['polarity'] = crypto['original_body'][ilist].apply(getPolarity)

In [145]:
# create a function get the sentiment text
def getSentiment(score):
    if score < 0:
        return "negative"
    elif score == 0:
        return "neutral"
    else:
        return "positive"

In [146]:
# create a column to store the text sentiment
crypto['sentiment'] = crypto['polarity'].apply(getSentiment)



In [185]:
def getpercentage(df):
    p = df.sentiment.loc[ df.sentiment == 'positive'].count()
    neg = df.sentiment.loc[ df.sentiment == 'negative'].count()
    n = df.sentiment.loc[ df.sentiment == 'neutral'].count()
    total = p + neg + n
    p1, p2, p3 = p/total, n/total, neg/total
    return [p1*100, p2*100, p3*100] 

In [186]:
df=crypto.loc[ilist]
values =getpercentage(df)
labels = ['Positive', 'Neutral', 'Negative']

In [187]:
values

[86.66666666666667, 0.0, 13.333333333333334]

In [191]:
colors = ['blue','grey','red']


In [192]:
import plotly.graph_objects as go
import plotly.express as px
fig6 = go.Figure(data = go.Pie(values = values, 
                               labels = labels, hole = 0.8,
                               marker_colors = colors ))
fig6.update_traces(hoverinfo='label+percent',
                   textinfo='percent', textfont_size=20)
fig6.add_annotation(x= 0.5, y = 0.5,
                    text = 'Sentiments',
                    font = dict(size=20,family='Verdana', 
                                color='black'),
                    showarrow = False)
fig6.show()


In [ ]:
st.pyplot(fig6)

In [ ]:
if __name__ == '__main__':
    main()
    import warnings
    warnings.filterwarnings('ignore')